## HOUSE PRICES SOLUTION - PART III APPLYING BEST MODEL TO TEST DATA
>* Import libraries and files
* Transform the test data as in the Part I of this project
* Apply Best Model (Ridge Regression over all features)
* Extract sales predictions for submission

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from pandas.tseries.offsets import *
import seaborn as sns
import re
from sklearn.preprocessing import Normalizer
import sqlite3
import os
from sklearn.linear_model import SGDRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import HuberRegressor
from scipy.linalg import lstsq # multiple linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy import stats

from sklearn import datasets, linear_model
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import QuantileTransformer
from sklearn.cluster import KMeans

import warnings
warnings.simplefilter("ignore")

In [2]:
# display all columns
pd.set_option('display.max_columns', None)

#### Load data (features and target data) from Part II to compare the structure with the test data and perform the prediction

In [3]:
# Load features data from file which did support the modeling in Part II
hp_model_PII = pd.read_csv('train_features_m3a_new.csv')

In [4]:
# remove the Unnamed column
hp_model_PII.drop('Unnamed: 0', axis=1, inplace=True)
hp_model_PII.shape

(1207, 210)

In [5]:
# Load target data from file which did support the modeling in Part II
hp_model = pd.read_csv('C:/Users/fbaff/EPFL ML Python/3- Machine Learning 1/ML1 Project submission/HousePrice_model_new.csv')

In [6]:
# remove the Unnamed column
hp_model.drop('Unnamed: 0', axis=1, inplace=True)
hp_model.shape

(2414, 212)

In [7]:
# Select LogSalePrice only
target_PII=hp_model['SalePrice']

In [8]:
# Load data for the final prediction (submission)
hp_transform = pd.read_csv('C:/Users/fbaff/Documents/Data Science/EPFL/0_Datasets/ML1 - Lr project/house-prices/house-prices-test.csv')

In [9]:
hp_transform.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
0,2217,909279080,50,RL,NaN,11275,Pave,NaN,IR1,HLS,AllPub,Corner,Mod,Crawfor,Norm,Norm,1Fam,1.5Fin,6,7,1932,1950,Gable,CompShg,MetalSd,MetalSd,BrkFace,480.0,TA,TA,CBlock,TA,TA,Mn,Rec,297.0,LwQ,557.0,0.0,854.0,GasA,TA,Y,SBrkr,1096,895,0,1991,0.0,0.0,1,1,3,1,TA,7,Typ,1,Gd,Detchd,1977.0,Unf,2.0,432.0,TA,Fa,Y,0,0,19,0,0,0,NaN,NaN,NaN,0,3,2007,WD,Normal
1,837,907126050,20,RL,65.0,9757,Pave,NaN,Reg,Low,AllPub,Inside,Mod,CollgCr,Norm,Norm,1Fam,1Story,5,7,1994,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,PConc,TA,TA,No,ALQ,755.0,Unf,0.0,235.0,990.0,GasA,Ex,Y,SBrkr,990,0,0,990,1.0,0.0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1995.0,RFn,1.0,440.0,TA,TA,Y,66,0,0,0,92,0,NaN,NaN,NaN,0,10,2009,WD,Normal
2,2397,528144030,60,RL,86.0,11065,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,Stone,788.0,Gd,TA,PConc,Gd,TA,Mn,Unf,0.0,Unf,0.0,1085.0,1085.0,GasA,Ex,Y,SBrkr,1120,850,0,1970,0.0,0.0,2,1,3,1,Ex,8,Typ,1,Gd,BuiltIn,2006.0,Fin,3.0,753.0,TA,TA,Y,177,74,0,0,0,0,NaN,NaN,NaN,0,10,2006,New,Partial
3,1963,535452060,20,RL,70.0,7000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1960,2002,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,45.0,TA,TA,CBlock,TA,TA,No,Rec,588.0,Unf,0.0,422.0,1010.0,GasA,Ex,Y,SBrkr,1134,0,0,1134,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Attchd,1960.0,RFn,1.0,254.0,TA,TA,Y,0,16,0,0,0,0,NaN,MnWw,NaN,0,4,2007,WD,Family
4,306,911202100,50,C (all),66.0,8712,Pave,Pave,Reg,HLS,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1.5Fin,4,7,1900,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,Stone,TA,TA,Mn,Unf,0.0,Unf,0.0,859.0,859.0,GasA,Gd,Y,SBrkr,859,319,0,1178,0.0,0.0,1,0,2,1,TA,7,Typ,0,NaN,Detchd,1964.0,RFn,1.0,384.0,TA,TA,N,68,0,98,0,0,0,NaN,NaN,NaN,0,1,2010,WD,Abnorml


In [10]:
hp_transform.shape

(500, 81)

### Transform test data accoding to steps performed in the Part I

#### Step 1b Remove variables which do not have value at all

In [11]:
df1=hp_transform.dropna(how='all', axis=1)
df1.shape

(500, 81)

#### Step 1c Remove columns: Alley, PoolQC, Fence, Misc Feature

In [12]:
# Drop columns Alley, PoolQC, Fence, Misc Feature
hp_transform.drop('Alley', axis=1, inplace=True)
hp_transform.drop('Pool QC', axis=1, inplace=True)
hp_transform.drop('Fence', axis=1, inplace=True)
hp_transform.drop('Misc Feature', axis=1, inplace=True)

In [13]:
hp_transform.shape

(500, 77)

#### Step 1d Replace NaN in 'Lot Frontage' column by the median of the given column

In [14]:
hp_transform['Lot Frontage'].replace(np.NaN, hp_transform['Lot Frontage'].median(), inplace=True)

#### Step 1e Convert Garage Yr Blt into a categorical feature with 10 classes

In [15]:
hp_transform['Garage Modernity'] = pd.qcut(hp_transform['Garage Yr Blt'], q=10).cat.codes

In [16]:
hp_transform.drop('Garage Yr Blt', axis=1, inplace=True)

In [17]:
hp_transform.shape

(500, 77)

#### Step 1f Replace NaN in each columns with float data type by the median of the given column

In [18]:
# select the float columns
hp1_float = hp_transform.select_dtypes(include=[np.float64])
# replace NaN values by median
hp1_float.replace(np.NaN, hp1_float.median(), inplace=True)

#### Step 1g Replace NaN in each columns with integer data type by the median of the given column

In [19]:
# select the integer columns
hp1_int = hp_transform.select_dtypes(include=[np.int64, np.int8])
# replace NaN values by median
hp1_int.replace(np.NaN, hp1_int.median(), inplace=True)

#### Step 1h Replace NaN in each columns with object data type by a "fake class" named 'tagada' (except Bsmt Qual)

In [20]:
# select the object columns
hp1_obj = hp_transform.select_dtypes(exclude=[np.number])

In [21]:
# Replace NaN by No in Bsmt Qual to acknowledge that some houses do not have basement
hp1_obj['Bsmt Qual'].replace(np.NaN, 'No', inplace=True)

In [22]:
# Replace NaN by "fake class" tagada
hp1_obj.fillna('tagada', inplace=True)
hp1_obj.shape

(500, 39)

#### Step 1i a dataframe 'hp_noNaN' with all columns merged without NaN

In [23]:
# merge float and integer data types columns into 'hp_noNaN_num'
hp_noNaN_num=pd.merge(hp1_int,hp1_float, right_index=True, left_index=True)
hp_noNaN_num.shape

(500, 38)

In [24]:
# merge hp_noNaN_num with dataframe containing object data type; new consolidated dataframe named: hp_noNaN
hp_noNaN=pd.merge(hp_noNaN_num,hp1_obj, right_index=True, left_index=True)
hp_noNaN.shape

(500, 77)

In [25]:
# re-order the columns of the dataframe
hp_noNaN = hp_noNaN.reindex(sorted(hp_noNaN.columns), axis=1)

### Feature engineering according to Part I process

#### Create new features - Continuous data (based on existing ones; e.g. grouping)

In [26]:
# Rename the reference dataframe hp_no_outliers1
hp_new1=hp_noNaN

In [27]:
# Create new features
hp_new1['Tot_liv_SF']=hp_new1['Total Bsmt SF'] + hp_new1['Gr Liv Area'] + hp_new1['1st Flr SF'] + hp_new1['2nd Flr SF']
hp_new1['Tot_ext_SF'] =  hp_new1['Mas Vnr Area']+ hp_new1['Wood Deck SF']+ hp_new1['Open Porch SF']+ hp_new1['Enclosed Porch']+ hp_new1['3Ssn Porch']+ hp_new1['Screen Porch']+hp_new1['Pool Area']
hp_new1['Gr_tot_SF'] = hp_new1['Tot_liv_SF'] + hp_new1['Tot_ext_SF'] + hp_new1['Garage Area']
hp_new1['SF_ind']=hp_new1['Lot Area'] - hp_new1['Gr_tot_SF']
hp_new1['Remod_ind'] = (hp_new1['Yr Sold'] - hp_new1['Year Built']) - (hp_new1['Year Remod/Add']-hp_new1['Year Built'])
hp_new1.head()

,1st Flr SF,2nd Flr SF,3Ssn Porch,Bedroom AbvGr,Bldg Type,Bsmt Cond,Bsmt Exposure,Bsmt Full Bath,Bsmt Half Bath,Bsmt Qual,Bsmt Unf SF,BsmtFin SF 1,BsmtFin SF 2,BsmtFin Type 1,BsmtFin Type 2,Central Air,Condition 1,Condition 2,Electrical,Enclosed Porch,Exter Cond,Exter Qual,Exterior 1st,Exterior 2nd,Fireplace Qu,Fireplaces,Foundation,Full Bath,Functional,Garage Area,Garage Cars,Garage Cond,Garage Finish,Garage Modernity,Garage Qual,Garage Type,Gr Liv Area,Half Bath,Heating,Heating QC,House Style,Kitchen AbvGr,Kitchen Qual,Land Contour,Land Slope,Lot Area,Lot Config,Lot Frontage,Lot Shape,Low Qual Fin SF,MS SubClass,MS Zoning,Mas Vnr Area,Mas Vnr Type,Misc Val,Mo Sold,Neighborhood,Open Porch SF,Order,Overall Cond,Overall Qual,PID,Paved Drive,Pool Area,Roof Matl,Roof Style,Sale Condition,Sale Type,Screen Porch,Street,TotRms AbvGrd,Total Bsmt SF,Utilities,Wood Deck SF,Year Built,Year Remod/Add,Yr Sold,Tot_liv_SF,Tot_ext_SF,Gr_tot_SF,SF_ind,Remod_ind
0,1096,895,0,3,1Fam,TA,Mn,0.0,0.0,TA,0.0,297.0,557.0,Rec,LwQ,Y,Norm,Norm,SBrkr,19,TA,TA,MetalSd,MetalSd,Gd,1,CBlock,1,Typ,432.0,2.0,Fa,Unf,4,TA,Detchd,1991,1,GasA,TA,1.5Fin,1,TA,HLS,Mod,11275,Corner,69.0,IR1,0,50,RL,480.0,BrkFace,0,3,Crawfor,0,2217,7,6,909279080,Y,0,CompShg,Gable,Normal,WD,0,Pave,7,854.0,AllPub,0,1932,1950,2007,4836.0,499.0,5767.0,5508.0,57
1,990,0,0,3,1Fam,TA,No,1.0,0.0,TA,235.0,755.0,0.0,ALQ,Unf,Y,Norm,Norm,SBrkr,0,Gd,TA,VinylSd,VinylSd,tagada,0,PConc,1,Typ,440.0,1.0,TA,RFn,5,TA,Attchd,990,0,GasA,Ex,1Story,1,TA,Low,Mod,9757,Inside,65.0,Reg,0,20,RL,0.0,None,0,10,CollgCr,0,837,7,5,907126050,Y,0,CompShg,Gable,Normal,WD,92,Pave,5,990.0,AllPub,66,1994,1994,2009,2970.0,158.0,3568.0,6189.0,15
2,1120,850,0,3,1Fam,TA,Mn,0.0,0.0,Gd,1085.0,0.0,0.0,Unf,Unf,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,Gd,1,PConc,2,Typ,753.0,3.0,TA,Fin,8,TA,BuiltIn,1970,1,GasA,Ex,2Story,1,Ex,Lvl,Gtl,11065,Inside,86.0,IR1,0,60,RL,788.0,Stone,0,10,NridgHt,74,2397,5,8,528144030,Y,0,CompShg,Gable,Partial,New,0,Pave,8,1085.0,AllPub,177,2006,2006,2006,5025.0,1039.0,6817.0,4248.0,0
3,1134,0,0,2,1Fam,TA,No,0.0,0.0,TA,422.0,588.0,0.0,Rec,Unf,Y,Norm,Norm,SBrkr,0,TA,TA,Wd Sdng,Wd Sdng,tagada,0,CBlock,1,Typ,254.0,1.0,TA,RFn,2,TA,Attchd,1134,0,GasA,Ex,1Story,1,TA,Lvl,Gtl,7000,Inside,70.0,Reg,0,20,RL,45.0,BrkFace,0,4,NAmes,16,1963,7,5,535452060,Y,0,CompShg,Gable,Family,WD,0,Pave,6,1010.0,AllPub,0,1960,2002,2007,3278.0,61.0,3593.0,3407.0,5
4,859,319,0,2,1Fam,TA,Mn,0.0,0.0,TA,859.0,0.0,0.0,Unf,Unf,Y,Norm,Norm,SBrkr,98,TA,TA,MetalSd,MetalSd,tagada,0,Stone,1,Typ,384.0,1.0,TA,RFn,2,TA,Detchd,1178,0,GasA,Gd,1.5Fin,1,TA,HLS,Mod,8712,Inside,66.0,Reg,0,50,C (all),0.0,None,0,1,IDOTRR,0,306,7,4,911202100,N,0,CompShg,Gable,Abnorml,WD,0,Pave,7,859.0,AllPub,68,1900,1950,2010,3215.0,166.0,3765.0,4947.0,60


### Removal of continuous features

In [28]:
# Remove features used for Tot_liv_SF
remove = ['Total Bsmt SF','Gr Liv Area','1st Flr SF','2nd Flr SF']
hp_new1.drop(remove, inplace=True, axis=1)

In [29]:
# Remove features used for Tot_ext_SF
remove1 = ['Mas Vnr Area','Wood Deck SF','Open Porch SF','Enclosed Porch','3Ssn Porch','Screen Porch','Pool Area']
hp_new1.drop(remove1, inplace=True, axis=1)

In [30]:
# Remove features with very low number of data points
remove2 = ['Low Qual Fin SF','Misc Val']
hp_new1.drop(remove2, inplace=True, axis=1)

In [31]:
# Remove features which are included in an existing total
remove3= ['Bedroom AbvGr', 'Kitchen AbvGr']
hp_new1.drop(remove3, inplace=True, axis=1)

In [32]:
hp_new1.shape

(500, 67)

### Transformation of features

In [33]:
# Add a Log_LotArea variable to the DataFrame
hp_new1['Log_LotArea'] = np.log10(hp_new1['Lot Area'])

In [34]:
hp_new1.shape

(500, 68)

### Encode ordinal and caregorical features

####  Encode ordinal features
* ordinal features will be encoded with Scikitlearn LabelEncoder() as we want to keep the ranking

In [35]:
# select the object columns
hp_new1_obj = hp_new1.select_dtypes(exclude=[np.number])
hp_new1_obj.shape

(500, 39)

In [36]:
# Create a dataframe with ordinal features
hp_new1_ordi=hp_new1[['Lot Shape','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Heating QC','Electrical',
                     'Kitchen Qual','Functional','Fireplace Qu','Garage Finish','Garage Qual',
                     'Garage Cond','Paved Drive', 'Exter Qual', 'Exter Cond', 'Utilities', 'Land Slope']]

In [37]:
hp_new1_ordi = hp_new1_ordi.replace({"Bsmt Cond" : {"tagada" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Bsmt Exposure" : {"tagada":0, "No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                                     "BsmtFin Type 1" : {"tagada" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                                     "BsmtFin Type 2" : {"tagada" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                                     "Bsmt Qual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                                     "Exter Cond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                                     "Exter Qual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                                     "Fireplace Qu" : {"tagada":0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8},
                                     "Garage Cond" : {"tagada" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Garage Qual" : {"tagada" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Heating QC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Kitchen Qual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                                     "Land Slope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                                     "Lot Shape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                                     "Paved Drive" : {"N" : 0, "P" : 1, "Y" : 2}, "Street" : {"Grvl" : 1, "Pave" : 2},
                                     "Electrical" : {"tagada":0, "Mix":0, "FuseP":1,"FuseF":2,"FuseA":3,"SBrkr":4},
                                     "Garage Finish" : {"tagada" : 0, "Unf" : 1, "RFn":2,"Fin":3},
                                     "Utilities" : {"NoSeWa" : 1, "NoSewr" : 2, "AllPub" : 3}})

In [38]:
hp_new1_ordi.head()

,Lot Shape,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin Type 2,Heating QC,Electrical,Kitchen Qual,Functional,Fireplace Qu,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Exter Qual,Exter Cond,Utilities,Land Slope
0,3,3,3,1,3,2,3,4,3,8,4,1,3,2,2,3,3,3,2
1,4,3,3,0,5,1,5,4,3,8,0,2,3,3,2,3,4,3,2
2,3,4,3,1,1,1,5,4,5,8,4,3,3,3,2,4,3,3,3
3,4,3,3,0,3,1,5,4,3,8,0,2,3,3,2,3,3,3,3
4,4,3,3,1,1,1,4,4,3,8,0,2,3,3,0,3,3,3,2


#### Encode nominal features

In [39]:
hp_new1_norm=hp_new1_obj.drop(columns=['Lot Shape','Bsmt Qual','Bsmt Cond','Bsmt Exposure','BsmtFin Type 1','BsmtFin Type 2','Heating QC','Electrical',
                     'Kitchen Qual','Functional','Fireplace Qu','Garage Finish','Garage Qual',
                     'Garage Cond','Paved Drive', 'Exter Qual', 'Exter Cond', 'Utilities', 'Land Slope'])

In [40]:
hp_new1_norm.shape

(500, 20)

In [41]:
# encode
encode_norm= pd.get_dummies(hp_new1_norm)

In [42]:
encode_norm.shape

(500, 138)

### Consolidate dataframe with numerical and transformed categorical features

In [43]:
# Create a data frame with with numerical data
hp_num_int = hp_new1.select_dtypes(include=[np.int64, np.float64])
hp_num_int.shape

(500, 28)

In [44]:
# transform float64 into int64 in case of dtype need for classification
hp_num_int['Bsmt Full Bath']=hp_num_int['Bsmt Full Bath'].astype(np.int64)
hp_num_int['Bsmt Half Bath']=hp_num_int['Bsmt Half Bath'].astype(np.int64)
hp_num_int['Bsmt Unf SF']=hp_num_int['Bsmt Unf SF'].astype(np.int64)
hp_num_int['BsmtFin SF 1']=hp_num_int['BsmtFin SF 1'].astype(np.int64)
hp_num_int['BsmtFin SF 2']=hp_num_int['BsmtFin SF 2'].astype(np.int64)
hp_num_int['Garage Area']=hp_num_int['Garage Area'].astype(np.int64)
hp_num_int['Garage Cars']=hp_num_int['Garage Cars'].astype(np.int64)
hp_num_int['Lot Frontage']=hp_num_int['Lot Frontage'].astype(np.int64)
hp_num_int['Tot_liv_SF']=hp_num_int['Tot_liv_SF'].astype(np.int64)
hp_num_int['Tot_ext_SF']=hp_num_int['Tot_ext_SF'].astype(np.int64)
hp_num_int['SF_ind']=hp_num_int['SF_ind'].astype(np.int64)
hp_num_int['Log_LotArea']=hp_num_int['Log_LotArea'].astype(np.int64)
hp_num_int['Gr_tot_SF']=hp_num_int['Gr_tot_SF'].astype(np.int64)

In [45]:
# Merge encoded features data
encode_tot=pd.merge(hp_new1_ordi,encode_norm, right_index=True, left_index=True)
encode_tot.shape

(500, 157)

In [46]:
# Merge numerical data and transformed categorical data
hp_model_PIII=pd.merge(hp_num_int,encode_tot, right_index=True, left_index=True)
hp_model_PIII.shape

(500, 185)

In [47]:
hp_model_PIII.head()

,Bsmt Full Bath,Bsmt Half Bath,Bsmt Unf SF,BsmtFin SF 1,BsmtFin SF 2,Fireplaces,Full Bath,Garage Area,Garage Cars,Half Bath,Lot Area,Lot Frontage,MS SubClass,Mo Sold,Order,Overall Cond,Overall Qual,PID,TotRms AbvGrd,Year Built,Year Remod/Add,Yr Sold,Tot_liv_SF,Tot_ext_SF,Gr_tot_SF,SF_ind,Remod_ind,Log_LotArea,Lot Shape,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin Type 2,Heating QC,Electrical,Kitchen Qual,Functional,Fireplace Qu,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Exter Qual,Exter Cond,Utilities,Land Slope,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,Central Air_N,Central Air_Y,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosN,Exterior 1st_AsbShng,Exterior 1st_BrkFace,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsbShng,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_tagada,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,MS Zoning_C (all),MS Zoning_FV,MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_None,Mas Vnr Type_Stone,Mas Vnr Type_tagada,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Roof Matl_CompShg,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial,Sale Type_COD,Sale Type_CWD,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_WD,Street_Grvl,Street_Pave
0,0,0,0,297,557,1,1,432,2,1,11275,69,50,3,2217,7,6,909279080,7,1932,1950,2007,4836,499,5767,5508,57,4,3,3,3,1,3,2,3,4,3,8,4,1,3,2,2,3,3,3,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1,1,0,235,755,0,0,1,440,1,0,9757,65,20,10,837,7,5,907126050,5,1994,1994,2009,2970,158,3568,6189,15,3,4,3,3,0,5,1,5,4,3,8,0,2,3,3,2,3,4,3,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
2,0,0,1085,0,0,1,2,753,3,1,11065,86,60,10,2397,5,8,528144030,8,2006,2006,2006,5025,1039,681

#### Compare the number of features between the 2 dataframes (model built in Part II vs Test file for making predictions)

In [48]:
hp_model_PII.shape

(1207, 210)

In [49]:
train_features = set(hp_model_PII.columns)
new_data_features = set(hp_model_PIII.columns)

In [50]:
# New features
new_data_features - train_features

{'Exterior 2nd_Other', 'Roof Matl_Roll'}

In [51]:
# Missing features from the train set
train_features - new_data_features

{'Condition 2_Artery',
 'Condition 2_PosA',
 'Condition 2_RRAe',
 'Condition 2_RRAn',
 'Condition 2_RRNn',
 'Exterior 1st_AsphShn',
 'Exterior 1st_BrkComm',
 'Exterior 1st_CBlock',
 'Exterior 1st_ImStucc',
 'Exterior 1st_PreCast',
 'Exterior 1st_Stone',
 'Exterior 2nd_AsphShn',
 'Exterior 2nd_PreCast',
 'Foundation_Wood',
 'Heating_Floor',
 'MS Zoning_A (agr)',
 'MS Zoning_I (all)',
 'Mas Vnr Type_CBlock',
 'Neighborhood_Greens',
 'Neighborhood_GrnHill',
 'Neighborhood_Landmrk',
 'Roof Matl_Membran',
 'Roof Matl_Metal',
 'Roof Matl_WdShngl',
 'Roof Style_Shed',
 'Sale Type_Con',
 'Sale Type_VWD'}

In [52]:
new_data_reindexed = hp_model_PIII.reindex(
    columns=hp_model_PII.columns,
    fill_value=0 # Fill with 0s
)
print('DataFrame shape:', new_data_reindexed.shape)

DataFrame shape: (500, 210)


In [53]:
new_data=pd.DataFrame(new_data_reindexed)
new_data.shape

(500, 210)

### Apply Best Model from Part II (Modeling)
* Best Model is: Ridge regression overl All Features with an Alpha value of 11.497569953977356

In [54]:
# Import train_target_PII data set
train_target_PII = pd.read_csv('C:/Users/fbaff/EPFL ML Python/3- Machine Learning 1/ML1 Project submission/train_target_PII.csv')

In [55]:
train_target_PII.shape

(1207, 2)

In [56]:
# remove the Unnamed column
train_target_PII.drop('Unnamed: 0', axis=1, inplace=True)
train_target_PII.shape

(1207, 1)

In [57]:
train_target_PII.head()

,LogSalePrice
0,5.117271
1,5.110590
2,5.269279
3,5.474071
4,5.117271


In [58]:
hp_model_PII.shape

(1207, 210)

In [59]:
# Create X and y variables for intermediate model
X_ridge=hp_model_PII.values
y_ridge=train_target_PII.values

In [60]:
# Fit Ridge Regression Best Alpha
ridge3 = Ridge(11.497569953977356)
ridge3.fit(X_ridge, y_ridge)

Ridge(alpha=11.497569953977356, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [61]:
# compute prediction
prediction = 10**ridge3.predict(new_data_reindexed)

In [62]:
new_data['SalePrice']=prediction

In [63]:
new_data.head()

,Bsmt Full Bath,Bsmt Half Bath,Bsmt Unf SF,BsmtFin SF 1,BsmtFin SF 2,Fireplaces,Full Bath,Garage Area,Garage Cars,Half Bath,Lot Area,Lot Frontage,MS SubClass,Mo Sold,Order,Overall Cond,Overall Qual,PID,TotRms AbvGrd,Year Built,Year Remod/Add,Yr Sold,Tot_liv_SF,Tot_ext_SF,Gr_tot_SF,SF_ind,Remod_ind,Log_LotArea,Lot Shape,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin Type 2,Heating QC,Electrical,Kitchen Qual,Functional,Fireplace Qu,Garage Finish,Garage Qual,Garage Cond,Paved Drive,Exter Qual,Exter Cond,Utilities,Land Slope,Bldg Type_1Fam,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,Central Air_N,Central Air_Y,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Artery,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNn,Exterior 1st_AsbShng,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsbShng,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_tagada,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone,Mas Vnr Type_tagada,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Street_Grvl,Street_Pave,SalePrice
0,0,0,0,297,557,1,1,432,2,1,11275,69,50,3,2217,7,6,909279080,7,1932,1950,2007,4836,499,5767,5508,57,4,3,3,3,1,3,2,3,4,3,8,4,1,3,2,2,3,3,3,2,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

In [64]:
submission=new_data[['PID','SalePrice']]

#### Check data structure of the SalePrice and Prediction
* Mean and Standard deviation in reference data and test (prediction) are similar

In [65]:
target_PII=pd.DataFrame(target_PII)

In [66]:
target_PII.describe()

,SalePrice
count,2414.000000
mean,178801.731152
std,76535.580619
min,12789.000000
25%,129000.000000
50%,160000.000000
75%,213000.000000
max,615000.000000


In [67]:
submission=pd.DataFrame(submission)

In [68]:
submission[['SalePrice']].describe()

,SalePrice
count,5.000000e+02
mean,1.852022e+05
std,9.319247e+04
min,4.674059e+04
25%,1.290472e+05
50%,1.622720e+05
75%,2.142362e+05
max,1.172629e+06


### Save the prediction file

In [69]:
# Save data
# submission.to_csv('house-prices-pred3.csv', index=False, float_format='%.0f')